<a href="https://colab.research.google.com/github/Urzelhen/NLP_streamlit_GPT_QA/blob/main/pushkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Неделя 7: NLP

## Генерация текста с помощью модели **GPT**

https://www.kaggle.com/tuckerarrants/text-generation-with-huggingface-gpt2

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
!pip install -q transformers

In [53]:
import numpy as np
import pandas as pd
import re
import random

import torch
from tqdm.notebook import tqdm
import transformers
from torch.optim import AdamW

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [54]:
# Загружаем токенайзер модели
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

In [55]:
# Загружаем файл с текстом, на котором будем дообучать модель
import re
# with open('/content/drive/MyDrive/avidreaders.ru__evgeniy-onegin.txt', encoding='utf8') as f:
#     text_onegin = f.read()


# with open('/content/drive/MyDrive/Zov-Ktulhu txt.txt', encoding='utf8') as f:
#     text_ktulhu = f.read()

# with open('/content/drive/MyDrive/Izgonyaushiy', encoding='utf8') as f:
#     text_izgonyaushiy = f.read()

# with open('/content/drive/MyDrive/tired_blood', encoding='utf8') as f:
#     text_tired_blood = f.read()

# with open('/content/drive/MyDrive/morgen', encoding='utf8') as f:
#     text_morgen = f.read()

# with open('/content/drive/MyDrive/50_shades_of_grey', encoding='utf8') as f:
#     text_50_shades = f.read()

# with open('/content/drive/MyDrive/horoscope', encoding='utf8') as f:
#     text_horoscope = f.read()

# text = '' + text_morgen + text_horoscope
#  + text_tired_blood

with open('/content/drive/MyDrive/pushkin', encoding='utf8') as f:
    text = f.read()

# New Section

In [118]:
# print(text[1000:5000])

In [23]:
roman_numbers = 'IXLVM'
chapters = ["Глава первая", "Глава вторая", "Глава третья", "Глава четвертая", "Глава пятая", "Глава шестая", "Глава седьмая", "Глава восьмая", "I. Ужас в глине", "II. Рассказ инспектора Легресса", "III. Морское безумие"]
for chapter in chapters:
  text.replace(chapter, ' ')
text = text.translate(str.maketrans('', '', roman_numbers))
# text = text.translate(str.maketrans('', '', chapters))
text = text.replace("\n* * *\n", '\n')
text = re.sub('\n{2,}', '\n', text)
text = re.sub('[\d]', '', text)
text = text.replace("[]", '')

In [24]:
# токенизируем текст
tokens = tokenizer.encode(text, add_special_tokens=True)
tokens = np.array(tokens)
print(len(tokens))
tokens[:10]

45495


array([20377,  2299, 20361,  3384,  6304,   203,  2176,   354,   669,
       30618])

In [25]:
# разбиваем на train и test

l = len(tokens)//15
train = []
test = []
for i in range(15):
    if i%5 > 0:
        train.extend(tokens[i*l: (i+1)*l])
    else:
        test.extend(tokens[i*l: (i+1)*l])
train = np.array(train)
test = np.array(test)

print(len(tokens), len(train), len(test))

45495 36396 9099


In [26]:
# re.split('; |, ', string_to_split)

In [27]:
from transformers import GPT2LMHeadModel

# Эту модель просто подгружаем и не будем дообучать 
model_init = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)


# Эту модель подгрузим и далее обучим
model = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)

model.to(device);
model_init.to(device);

In [28]:
batch_size = 16
max_len = 256
epochs = 10

n_train = len(train)//(batch_size*max_len)
n_test = len(test)//(batch_size*max_len)
print(n_train, n_test)

# устанавливаем оптимизатор
optimizer = AdamW(model.parameters(), lr = 1e-5, eps = 1e-8)

# трансформеры с трудом обучаются, для них нужны разные способы повышения 
# эффективности градиентного спуска
total_steps = n_train * epochs
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def accuracy(y_true, logits):
    return torch.mean((y_true[1:] == torch.argmax(logits, dim=2)[:-1]).float()).detach().cpu().numpy()

8 2


In [29]:
# готовим тензоры для обучения, поскольку не пользуемся DataLoader

def prep_tensors(x, i, batch_size=batch_size, max_len=max_len):
    batch_ids = x[i*batch_size*max_len: (i+1)*batch_size*max_len]
    batch_ids = batch_ids.reshape(batch_size, max_len)
    batch_ids = torch.tensor(batch_ids).to(device)
    return batch_ids


# обучающий цикл
for epoch in range(1, epochs+1):
    print(f'epoch {epoch}/{epochs} : training')

    train_loss = []
    train_acc = []
    model.train()
    pbar = tqdm(range(n_train))
    for i in pbar:
        batch_ids = prep_tensors(train, i)

        model.zero_grad() # = optimizer.zero_grad()
        loss, logits, _ = model(batch_ids, # logits - что подаем на SoftMax
                                token_type_ids=None, 
                                
                                labels=batch_ids
                             ).values()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # нормируем веса, что норма была 1
        optimizer.step()
        scheduler.step()
        
        train_loss.append(loss.item())
        train_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(train_acc):.4f} loss {np.mean(train_loss):.4f}', refresh=True)

    
    print(f'epoch {epoch}/{epochs} : validation')
    model.eval()
    val_acc = []
    val_loss = []
    pbar = tqdm(range(n_test))
    for i in pbar:
        batch_ids = prep_tensors(test, i)
        with torch.no_grad():        
            loss, logits, _ = model(batch_ids, 
                                token_type_ids=None, 
                                # attention_mask=batch_mask,
                                labels=batch_ids
                                 ).values()
        
        val_loss.append(loss.item())
        val_acc.append(accuracy(batch_ids, logits))
        pbar.set_description(f'acc {np.mean(val_acc):.4f} loss {np.mean(val_loss):.4f}', refresh=True)


epoch 1/10 : training


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 1/10 : validation


  0%|          | 0/2 [00:00<?, ?it/s]

epoch 2/10 : training


...

  0%|          | 0/2 [00:00<?, ?it/s]

epoch 10/10 : training


  0%|          | 0/8 [00:00<?, ?it/s]

epoch 10/10 : validation


  0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
import textwrap

In [ ]:
# https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate
# модель без дообучения

# prompt = "print('Hello world!') "
prompt = 'У Лукоморья дуб зеленый'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_init.generate(
    input_ids=prompt,
    max_length=50,
    num_beams=5,
    do_sample=True,
    temperature=10.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=3,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 120), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


У Лукоморья дуб зеленый с розовым веночком. У меня вот так, по случаю рождения Белочек! В детстве его любила... Роза
Алена - у тебя какой год в году??? Я сейчас смотрю- у
------------------
У Лукоморья дуб зеленый. И не было такого дуба ни одного – все было как всегда, просто по-своему – просто дуб был один!
Вот это вот самое: «А дуб - это же дерево дуб, не дерево»!
------------------
У Лукоморья дуб зеленый.  И как мне не бояться дубка-сосновка?!   И все же дубу очень рад — в самом сердце Подмосковья
он словно в сердце своем, на глазах — зеленый, а вот дерево...
------------------
У Лукоморья дуб зеленый и кувшин. Я люблю свой дом Николай Петрович Шкуматов  Автор  «Кому нужна власть? Не нужен
никому. Никто. А если ты в нем есть и живешь, значит и нужен.
------------------
У Лукоморья дуб зеленый...   А ведь есть, оказывается: и люди-колобочки на белом поле   За окном метелицу и дожди и
ветер-холодец   А мы с тобою на белой, пушистой траве сидим,
------------------
У Лукоморья дуб зеле

In [35]:
# дообученная модель

prompt = 'Бродил котенок'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model.generate(
    input_ids=prompt,
    max_length=150,
    num_beams=5,
    do_sample=True,
    temperature=100.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=7,
    ).cpu().numpy()
for out_ in out:
    print(textwrap.fill(tokenizer.decode(out_), 150), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Бродил котенок на свободе. В руках держал кол, а сам в лукошки бросал: — Губки трубочкой зажала он и в угол заговорит — в окошко заглянешь. Вот опять
— все о покойной: сидит там с той стороны и на ухо мне бубенчит; глядят туда: она в окно точит — глядят на него с укоризнью: чего врешь! ну прямо, что
баба. Вот тогда в окне, на стене виснет карта. Тут Котяру увидел и кричит своим громким: вси, ви, барин; вись в лесу губе!.. Ну а как тут забиться
стало и не стало б котеночку — сам на нее прыгнул,
------------------
Бродил котенок и искал: "Не бегают-с белки на курочки;   Догони-я найду. За что меня?" А сам подумал да, так ведь он котенку рад. "Отнеси мне домой".
А сам опять вспомнил; — "Зачем, где мой серый красавец? Что делать? Как ты смел бы ему угодить — он в доме сидеть да не ходить, не сидеть! Нет!" —
сказал и понес серого красава за куст — и принес его к нам за обедом, чтобы показать — да нет такого: как ты сам догадался его увидеть? И как? Все так
хорошо у тебя; даже кошка — она вс

In [36]:
out

array([[  983,   679,   364, ...,  1710, 31245,    16],
       [  983,   679,   364, ...,    16,  4750,    16],
       [  983,   679,   364, ...,  1176, 13803,   318],
       ...,
       [  983,   679,   364, ...,  1336,   694,    16],
       [  983,   679,   364, ...,  5974,    16,  5974],
       [  983,   679,   364, ..., 28525, 11624,   723]])

In [ ]:
# def generate(prompt, len_gen=20, temperature=1):
#     generated = tokenizer.encode(prompt)
#     context = torch.tensor([generated]).to(device)
#     past = None

#     for i in tqdm(range(len_gen)):
#         output, past = model(context, past_key_values=past).values()
#         # token = torch.argmax(output[..., -1, :], dim=-1)
#         output = output / temperature
#         token = torch.distributions.Categorical(logits=output[..., -1, :]).sample()
        
#         generated += token.tolist()
#         context = token.unsqueeze(0)

#     sequence = tokenizer.decode(generated)

#     return sequence

In [37]:
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_faitytales')

In [47]:
model_2 = GPT2LMHeadModel.from_pretrained(
    'sberbank-ai/rugpt3small_based_on_gpt2',
    output_attentions = False,
    output_hidden_states = False,
)
# torch.load(model_2, '/content/drive/MyDrive/weights_faitytales')
# model_2 = load_state_dict(torch.load(('/content/drive/MyDrive/weights_faitytales')))
model_2.load_state_dict(torch.load('/content/drive/MyDrive/weights_faitytales'))

model_2.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [120]:
prompt = 'Жили-были'
prompt = tokenizer.encode(prompt, return_tensors='pt').to(device)
out = model_2.generate(
    input_ids=prompt,
    max_length=100,
    num_beams=5,
    do_sample=True,
    temperature=100.,
    top_k=50,
    top_p=0.6,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    ).cpu().numpy()
for out_ in out:
    end_of_sentence = [tokenizer.decode(out_).rfind("."), tokenizer.decode(out_).rfind("?"), tokenizer.decode(out_).rfind("!"), tokenizer.decode(out_).rfind('"'), tokenizer.decode(out_).rfind('...')]
    print(textwrap.fill(tokenizer.decode(out_)[:max(end_of_sentence)+1], 150), end='\n------------------\n')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Жили-были трое. Но когда у всех подросли два богатыра; Однажды к одной великой горе пришли четыре девицы и давай спрашивать о жизни и об одном народе.
А та, одна красавица: в том ли селе был старик богатырь-староста? У него в руке два ведра молока пил — он и рассказал своей жене. — «Не говори ему
ничего и ешь из этих рук, что пить из этого, я за себя поручу — так ты и сам ешь!
------------------
Жили-были две рыбейки и две гурии…» И все им отвечали «Простите меня! Но если так хотите и быть довольны; Я — один; а гурия одна! Вот вам на здоровье.
Да разве в таком мире не лучше ль жить?.. И тут уж рыбки на том остановились: да зачем? Ведь теперь в мире и так хорошо, — и, поутра немного, да к
гурие поближе Ходил бы в своем добром плаще».
------------------
Жили-были люди и цари в России». «Ты мне веришь ли»?» И тут «Я», и потом все с нею сделалось; И вот он опять идет и на улице, на которой жил… — В чем,
и что это за место было в этом ряду, — и как? «Где? у себя, вот и деревня есть.
-------